In [1]:
import keypress_recognition.models as models
from keypress_recognition import dataset_p as dataset
from keypress_recognition import dataset_t
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

In [2]:
concat = False
delta = 3 if concat else 1
channels = 3 * delta

In [3]:
mod = {}
mod['black'] = models.get_black_key_model(channels)
mod['black'].load_model('keypress_recognition/models/best_black_model.tar')
mod['white'] = models.get_white_key_model(channels)
mod['white'].load_model('keypress_recognition/models/best_white_model.tar') 

Trying to load model from keypress_recognition/models/best_black_model.tar ...
done
Trying to load model from keypress_recognition/models/best_white_model.tar ...
done


In [ ]:
dataset.load_all_data(color=['black', 'white'], size=['single'], keypress=True, concatenate=concat, delta=delta)

Start seperating keyboard ...
  White single width: 21px
  Black single width: 20px
  White bundle width: 37px
  Black bundle width: 36px


IntProgress(value=0, max=14273)

In trainset: 
  # of pressed black key: 8755
  # of unpressed black key: 43949
  # of pressed white key: 19083
  # of unpressed white key: 57523


IntProgress(value=0, max=5000)

In testset: 
  # of pressed black key: 761
  # of unpressed black key: 22436
  # of pressed white key: 10831
  # of unpressed white key: 22917


IntProgress(value=0, max=2993)

In [ ]:
def visualize_keys(note):
    """
    note: bool
    return: str
    """
    return ("Y" if note else "N")

def display_concat_img(img, idx=None):
    if concat:
        if idx is None:
            return img[:, :, :, 3:6]
        else:
            return img[idx, :, :, 3:6]
    if idx is None:
        return img
    else:
        return img[idx, :, :, :]


In [ ]:
for color in ('black', 'white'):
    results = mod[color].test(dataset, size='single', color=color, concatenate=concat)
    for k, v in results.items():
        print("{:10}: {:.4f}".format(k.title(), v))

In [4]:
npy = np.empty((0, 88), dtype=bool)
for whites, blacks in dataset_t.data_batch(size='single'):
#     print(blacks.shape, whites.shape)
    blackout = mod['black'].evaluate(blacks).numpy()
    whiteout = mod['white'].evaluate(whites).numpy()
#     print(blackout.shape, whiteout.shape)
#     print(blackout.dtype, whiteout.dtype)
    out = dataset_t.merge_two_colors(whiteout, blackout, dtype=bool)
    npy = np.concatenate((npy, out[None]), axis=0)
#     np.save(f'{color}_prediction.npy', pred[color])

IntProgress(value=0, max=5000)

In [5]:
print(npy.shape)
np.save('keypress_recognition/dataset/y_pred/y_pred_single.npy', npy)
# print(blackout)
# print(whiteout)
# print(npy)
# print(npy[0, np.array([27, 29, 31, 34])])

(5000, 88)


In [6]:
# sanity check
label = np.load('keypress_recognition/dataset/y_test/y_test.npy')
label = label[:, 21:109] > 0
print(label.shape)
eva = dataset_t.evaluate(npy, label)
print(eva)

(5000, 88)
{'accuracy': 0.9331522727272727, 'precision': 0.08375764936702948, 'recall': 0.15467563837129056, 'F1 score': 0.10866995969574837}


In [7]:
black_mask = np.array([1, 4, 6, 9, 11, 13, 16, 18, 21, 23, 25, 28, 30, 33, 35, 37, 40, 42, 45, 47, 49, 52, 54, 57, 59, 61, 64,
    66, 69, 71, 73, 76, 78, 81, 83, 85])
white_mask = np.array([0, 2, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 20, 22, 24, 26, 27, 29, 31, 32, 34, 36, 38, 39, 41, 43, 44, 46, 48,
    50, 51, 53, 55, 56, 58, 60, 62, 63, 65, 67, 68, 70, 72, 74, 75, 77, 79, 80, 82, 84, 86, 87])
print(dataset_t.evaluate(npy[:, black_mask], label[:, black_mask]))
print(dataset_t.evaluate(npy[:, white_mask], label[:, white_mask]))

{'accuracy': 0.9830166666666666, 'precision': 0.008561643835616438, 'recall': 0.026281208935611037, 'F1 score': 0.012915724895059733}
{'accuracy': 0.8986307692307692, 'precision': 0.09296838131193959, 'recall': 0.16369679623303482, 'F1 score': 0.11858738545916661}
